In [1]:
import tensorflow as tf
from tensorflow import keras

import IPython

In [2]:
!pip install -q -U keras-tuner
import kerastuner as kt

In [4]:
(img_train, label_train), (img_test, label_test) = keras.datasets.fashion_mnist.load_data()


In [5]:
# Normalize pixel values between 0 and 1
img_train = img_train.astype('float32') / 255.0
img_test = img_test.astype('float32') / 255.0

In [6]:
def model_builder(hp):
  model = keras.Sequential()
  model.add(keras.layers.Flatten(input_shape=(28, 28)))
  
  # Tune the number of units in the first Dense layer
  # Choose an optimal value between 32-512
  hp_units = hp.Int('units', min_value = 32, max_value = 512, step = 32)
  model.add(keras.layers.Dense(units = hp_units, activation = 'relu'))
  model.add(keras.layers.Dense(10))

  # Tune the learning rate for the optimizer 
  # Choose an optimal value from 0.01, 0.001, or 0.0001
  hp_learning_rate = hp.Choice('learning_rate', values = [1e-2, 1e-3, 1e-4]) 
  
  model.compile(optimizer = keras.optimizers.Adam(learning_rate = hp_learning_rate),
                loss = keras.losses.SparseCategoricalCrossentropy(from_logits = True), 
                metrics = ['accuracy'])
  
  return model

In [9]:
tuner = kt.Hyperband(model_builder,
                     objective = 'val_accuracy', 
                     max_epochs = 10,
                     factor = 3,
                     directory = 'my_dir',
                     project_name = 'intro_to_kt')                       

In [8]:
class ClearTrainingOutput(tf.keras.callbacks.Callback):
  def on_train_end(*args, **kwargs):
    IPython.display.clear_output(wait = True)

In [10]:
tuner.search(img_train, label_train, epochs = 10, validation_data = (img_test, label_test), callbacks = [ClearTrainingOutput()])

# Get the optimal hyperparameters
best_hps = tuner.get_best_hyperparameters(num_trials = 1)[0]

print(f"""
The hyperparameter search is complete. The optimal number of units in the first densely-connected
layer is {best_hps.get('units')} and the optimal learning rate for the optimizer
is {best_hps.get('learning_rate')}.
""")

INFO:tensorflow:Oracle triggered exit

The hyperparameter search is complete. The optimal number of units in the first densely-connected
layer is 384 and the optimal learning rate for the optimizer
is 0.001.



In [11]:
# Build the model with the optimal hyperparameters and train it on the data
model = tuner.hypermodel.build(best_hps)
model.fit(img_train, label_train, epochs = 10, validation_data = (img_test, label_test))

Epoch 1/10


1554/1875 [=======================>......] - ETA: 0s - loss: 2.6626 - accuracy: 0.03 - ETA: 5s - loss: 1.5904 - accuracy: 0.45 - ETA: 6s - loss: 1.2418 - accuracy: 0.57 - ETA: 6s - loss: 1.0852 - accuracy: 0.61 - ETA: 6s - loss: 0.9862 - accuracy: 0.64 - ETA: 6s - loss: 0.9371 - accuracy: 0.66 - ETA: 6s - loss: 0.8904 - accuracy: 0.68 - ETA: 6s - loss: 0.8534 - accuracy: 0.70 - ETA: 6s - loss: 0.8284 - accuracy: 0.70 - ETA: 6s - loss: 0.8092 - accuracy: 0.71 - ETA: 6s - loss: 0.7954 - accuracy: 0.71 - ETA: 6s - loss: 0.7802 - accuracy: 0.72 - ETA: 6s - loss: 0.7669 - accuracy: 0.72 - ETA: 6s - loss: 0.7535 - accuracy: 0.73 - ETA: 6s - loss: 0.7467 - accuracy: 0.73 - ETA: 7s - loss: 0.7410 - accuracy: 0.73 - ETA: 7s - loss: 0.7353 - accuracy: 0.73 - ETA: 7s - loss: 0.7324 - accuracy: 0.74 - ETA: 7s - loss: 0.7274 - accuracy: 0.74 - ETA: 7s - loss: 0.7237 - accuracy: 0.74 - ETA: 7s - loss: 0.7178 - accuracy: 0.74 - ETA: 7s - loss: 0.7152 - accuracy: 0.74 - ETA: 7s - loss: 0.7095 - accura

1875/1875 [==============================] - ETA: 2s - loss: 0.4944 - accuracy: 0.82 - ETA: 2s - loss: 0.4940 - accuracy: 0.82 - ETA: 2s - loss: 0.4937 - accuracy: 0.82 - ETA: 2s - loss: 0.4932 - accuracy: 0.82 - ETA: 1s - loss: 0.4927 - accuracy: 0.82 - ETA: 1s - loss: 0.4918 - accuracy: 0.82 - ETA: 1s - loss: 0.4912 - accuracy: 0.82 - ETA: 1s - loss: 0.4909 - accuracy: 0.82 - ETA: 1s - loss: 0.4905 - accuracy: 0.82 - ETA: 1s - loss: 0.4901 - accuracy: 0.82 - ETA: 1s - loss: 0.4894 - accuracy: 0.82 - ETA: 1s - loss: 0.4888 - accuracy: 0.82 - ETA: 1s - loss: 0.4879 - accuracy: 0.82 - ETA: 1s - loss: 0.4878 - accuracy: 0.82 - ETA: 1s - loss: 0.4869 - accuracy: 0.82 - ETA: 1s - loss: 0.4864 - accuracy: 0.82 - ETA: 1s - loss: 0.4856 - accuracy: 0.82 - ETA: 1s - loss: 0.4855 - accuracy: 0.82 - ETA: 1s - loss: 0.4857 - accuracy: 0.82 - ETA: 0s - loss: 0.4853 - accuracy: 0.82 - ETA: 0s - loss: 0.4852 - accuracy: 0.82 - ETA: 0s - loss: 0.4850 - accuracy: 0.82 - ETA: 0s - loss: 0.4846 - accura

1793/1875 [===========================>..] - ETA: 0s - loss: 0.6457 - accuracy: 0.68 - ETA: 8s - loss: 0.4635 - accuracy: 0.83 - ETA: 9s - loss: 0.4267 - accuracy: 0.85 - ETA: 8s - loss: 0.4075 - accuracy: 0.85 - ETA: 8s - loss: 0.4080 - accuracy: 0.86 - ETA: 8s - loss: 0.3904 - accuracy: 0.86 - ETA: 7s - loss: 0.3880 - accuracy: 0.86 - ETA: 7s - loss: 0.3822 - accuracy: 0.86 - ETA: 8s - loss: 0.3834 - accuracy: 0.86 - ETA: 8s - loss: 0.3862 - accuracy: 0.86 - ETA: 9s - loss: 0.3904 - accuracy: 0.86 - ETA: 9s - loss: 0.3905 - accuracy: 0.85 - ETA: 9s - loss: 0.3916 - accuracy: 0.85 - ETA: 10s - loss: 0.3861 - accuracy: 0.859 - ETA: 10s - loss: 0.3883 - accuracy: 0.859 - ETA: 10s - loss: 0.3846 - accuracy: 0.860 - ETA: 10s - loss: 0.3813 - accuracy: 0.860 - ETA: 10s - loss: 0.3810 - accuracy: 0.859 - ETA: 10s - loss: 0.3795 - accuracy: 0.861 - ETA: 10s - loss: 0.3802 - accuracy: 0.859 - ETA: 9s - loss: 0.3777 - accuracy: 0.861 - ETA: 9s - loss: 0.3748 - accuracy: 0.86 - ETA: 9s - loss: 

1875/1875 [==============================] - ETA: 0s - loss: 0.3615 - accuracy: 0.86 - ETA: 0s - loss: 0.3614 - accuracy: 0.86 - ETA: 0s - loss: 0.3613 - accuracy: 0.86 - ETA: 0s - loss: 0.3613 - accuracy: 0.86 - ETA: 0s - loss: 0.3614 - accuracy: 0.86 - ETA: 0s - loss: 0.3616 - accuracy: 0.86 - 11s 6ms/step - loss: 0.3613 - accuracy: 0.8687 - val_loss: 0.3674 - val_accuracy: 0.8634
Epoch 3/10
1875/1875 [==============================] - ETA: 0s - loss: 0.1576 - accuracy: 0.90 - ETA: 18s - loss: 0.2612 - accuracy: 0.885 - ETA: 23s - loss: 0.2725 - accuracy: 0.892 - ETA: 17s - loss: 0.2946 - accuracy: 0.878 - ETA: 18s - loss: 0.3151 - accuracy: 0.876 - ETA: 17s - loss: 0.3467 - accuracy: 0.875 - ETA: 15s - loss: 0.3411 - accuracy: 0.875 - ETA: 16s - loss: 0.3352 - accuracy: 0.876 - ETA: 16s - loss: 0.3598 - accuracy: 0.866 - ETA: 16s - loss: 0.3538 - accuracy: 0.869 - ETA: 16s - loss: 0.3489 - accuracy: 0.870 - ETA: 16s - loss: 0.3497 - accuracy: 0.869 - ETA: 16s - loss: 0.3381 - accura

...

1875/1875 [==============================] - ETA: 0s - loss: 0.1457 - accuracy: 0.93 - ETA: 5s - loss: 0.3145 - accuracy: 0.88 - ETA: 5s - loss: 0.3047 - accuracy: 0.88 - ETA: 5s - loss: 0.2789 - accuracy: 0.89 - ETA: 5s - loss: 0.2803 - accuracy: 0.89 - ETA: 5s - loss: 0.2801 - accuracy: 0.89 - ETA: 5s - loss: 0.2717 - accuracy: 0.89 - ETA: 5s - loss: 0.2808 - accuracy: 0.89 - ETA: 5s - loss: 0.2742 - accuracy: 0.89 - ETA: 5s - loss: 0.2716 - accuracy: 0.90 - ETA: 5s - loss: 0.2686 - accuracy: 0.90 - ETA: 5s - loss: 0.2699 - accuracy: 0.90 - ETA: 5s - loss: 0.2693 - accuracy: 0.89 - ETA: 5s - loss: 0.2711 - accuracy: 0.89 - ETA: 5s - loss: 0.2725 - accuracy: 0.89 - ETA: 5s - loss: 0.2688 - accuracy: 0.89 - ETA: 5s - loss: 0.2684 - accuracy: 0.89 - ETA: 4s - loss: 0.2687 - accuracy: 0.89 - ETA: 4s - loss: 0.2678 - accuracy: 0.89 - ETA: 4s - loss: 0.2678 - accuracy: 0.90 - ETA: 4s - loss: 0.2652 - accuracy: 0.90 - ETA: 4s - loss: 0.2669 - accuracy: 0.90 - ETA: 4s - loss: 0.2686 - accura

1875/1875 [==============================] - ETA: 0s - loss: 0.6738 - accuracy: 0.78 - ETA: 5s - loss: 0.2673 - accuracy: 0.89 - ETA: 5s - loss: 0.2537 - accuracy: 0.91 - ETA: 5s - loss: 0.2421 - accuracy: 0.91 - ETA: 5s - loss: 0.2363 - accuracy: 0.91 - ETA: 5s - loss: 0.2485 - accuracy: 0.90 - ETA: 5s - loss: 0.2471 - accuracy: 0.90 - ETA: 5s - loss: 0.2544 - accuracy: 0.90 - ETA: 5s - loss: 0.2569 - accuracy: 0.90 - ETA: 5s - loss: 0.2551 - accuracy: 0.90 - ETA: 5s - loss: 0.2573 - accuracy: 0.90 - ETA: 5s - loss: 0.2530 - accuracy: 0.90 - ETA: 5s - loss: 0.2575 - accuracy: 0.90 - ETA: 5s - loss: 0.2543 - accuracy: 0.90 - ETA: 5s - loss: 0.2505 - accuracy: 0.90 - ETA: 5s - loss: 0.2507 - accuracy: 0.90 - ETA: 5s - loss: 0.2523 - accuracy: 0.90 - ETA: 4s - loss: 0.2516 - accuracy: 0.90 - ETA: 4s - loss: 0.2495 - accuracy: 0.90 - ETA: 4s - loss: 0.2485 - accuracy: 0.90 - ETA: 4s - loss: 0.2486 - accuracy: 0.90 - ETA: 4s - loss: 0.2469 - accuracy: 0.90 - ETA: 4s - loss: 0.2468 - accura

1875/1875 [==============================] - ETA: 0s - loss: 0.1613 - accuracy: 0.90 - ETA: 5s - loss: 0.2616 - accuracy: 0.90 - ETA: 5s - loss: 0.2620 - accuracy: 0.90 - ETA: 5s - loss: 0.2522 - accuracy: 0.90 - ETA: 5s - loss: 0.2512 - accuracy: 0.90 - ETA: 5s - loss: 0.2493 - accuracy: 0.90 - ETA: 5s - loss: 0.2397 - accuracy: 0.91 - ETA: 5s - loss: 0.2406 - accuracy: 0.91 - ETA: 5s - loss: 0.2440 - accuracy: 0.90 - ETA: 5s - loss: 0.2425 - accuracy: 0.90 - ETA: 5s - loss: 0.2424 - accuracy: 0.90 - ETA: 5s - loss: 0.2427 - accuracy: 0.90 - ETA: 5s - loss: 0.2440 - accuracy: 0.90 - ETA: 5s - loss: 0.2481 - accuracy: 0.90 - ETA: 5s - loss: 0.2463 - accuracy: 0.90 - ETA: 5s - loss: 0.2434 - accuracy: 0.91 - ETA: 5s - loss: 0.2437 - accuracy: 0.91 - ETA: 4s - loss: 0.2420 - accuracy: 0.91 - ETA: 4s - loss: 0.2413 - accuracy: 0.91 - ETA: 4s - loss: 0.2417 - accuracy: 0.91 - ETA: 4s - loss: 0.2433 - accuracy: 0.91 - ETA: 4s - loss: 0.2460 - accuracy: 0.90 - ETA: 4s - loss: 0.2472 - accura

1875/1875 [==============================] - ETA: 0s - loss: 0.3314 - accuracy: 0.84 - ETA: 7s - loss: 0.2760 - accuracy: 0.88 - ETA: 7s - loss: 0.2620 - accuracy: 0.89 - ETA: 7s - loss: 0.2259 - accuracy: 0.91 - ETA: 7s - loss: 0.2232 - accuracy: 0.91 - ETA: 7s - loss: 0.2202 - accuracy: 0.91 - ETA: 6s - loss: 0.2210 - accuracy: 0.91 - ETA: 6s - loss: 0.2246 - accuracy: 0.91 - ETA: 6s - loss: 0.2221 - accuracy: 0.91 - ETA: 7s - loss: 0.2229 - accuracy: 0.91 - ETA: 7s - loss: 0.2230 - accuracy: 0.91 - ETA: 7s - loss: 0.2264 - accuracy: 0.91 - ETA: 7s - loss: 0.2248 - accuracy: 0.91 - ETA: 7s - loss: 0.2274 - accuracy: 0.91 - ETA: 6s - loss: 0.2264 - accuracy: 0.91 - ETA: 6s - loss: 0.2280 - accuracy: 0.91 - ETA: 6s - loss: 0.2283 - accuracy: 0.91 - ETA: 6s - loss: 0.2286 - accuracy: 0.91 - ETA: 6s - loss: 0.2267 - accuracy: 0.91 - ETA: 6s - loss: 0.2285 - accuracy: 0.91 - ETA: 6s - loss: 0.2282 - accuracy: 0.91 - ETA: 6s - loss: 0.2273 - accuracy: 0.91 - ETA: 6s - loss: 0.2272 - accura

1875/1875 [==============================] - ETA: 0s - loss: 0.4487 - accuracy: 0.81 - ETA: 6s - loss: 0.2984 - accuracy: 0.87 - ETA: 6s - loss: 0.2455 - accuracy: 0.89 - ETA: 6s - loss: 0.2502 - accuracy: 0.89 - ETA: 6s - loss: 0.2319 - accuracy: 0.90 - ETA: 6s - loss: 0.2254 - accuracy: 0.90 - ETA: 6s - loss: 0.2301 - accuracy: 0.90 - ETA: 5s - loss: 0.2324 - accuracy: 0.90 - ETA: 5s - loss: 0.2264 - accuracy: 0.91 - ETA: 6s - loss: 0.2262 - accuracy: 0.91 - ETA: 6s - loss: 0.2363 - accuracy: 0.90 - ETA: 6s - loss: 0.2344 - accuracy: 0.90 - ETA: 6s - loss: 0.2371 - accuracy: 0.90 - ETA: 6s - loss: 0.2341 - accuracy: 0.90 - ETA: 6s - loss: 0.2343 - accuracy: 0.90 - ETA: 6s - loss: 0.2326 - accuracy: 0.91 - ETA: 6s - loss: 0.2308 - accuracy: 0.91 - ETA: 6s - loss: 0.2320 - accuracy: 0.91 - ETA: 6s - loss: 0.2313 - accuracy: 0.91 - ETA: 6s - loss: 0.2308 - accuracy: 0.91 - ETA: 6s - loss: 0.2314 - accuracy: 0.91 - ETA: 6s - loss: 0.2335 - accuracy: 0.91 - ETA: 6s - loss: 0.2346 - accura